# CITEseq data analysis

*Author: Lena Boehme, Taghon lab, 2023*

## Comparing differentiation pseudotime with spatial mapping

We have derived pseudotimes for developing CD4 and CD8 lineage cells. Furthermore, the dataset has been hyperclustered and then used for deconvolution of paediatric Visium data. By mapping the CMA values back to the CITE-seq data, we can now compare differentiation progress with thymic localisation

### Setup

In [ ]:
setwd("/home/lenab/Documents/scSeq_analyses/B_TotalThymus_CITEseq/2022_TotalThymus_CITEseq_HTA/objects")

In [ ]:
#default plotting settings

options(repr.plot.width=12, repr.plot.height=6)

options(scipen=100) #avoid scientific notation of numbers

In [ ]:
library(SeuratDisk)
library(Seurat)
library(matrixStats)
library(ggplot2)
library(pheatmap)
library(reshape2)
library(dplyr)
library(tidyr)
library(viridis)
library(RColorBrewer)
library(stringr)
library(batchelor)
library(BiocParallel)
library(BiocNeighbors)
library(slingshot)
library(tradeSeq)
library(SingleCellExperiment)
library(scater)
library(destiny)
library(pals)
library(ggrastr)
library(scales)

In [ ]:
sessionInfo()

In [ ]:
pal8 <- brewer.paired(8)
pal12 <- brewer.paired(12)
pal10 <- brewer.paired(10)
pal16 <- brewer.paired(16)
pal10v2 <- pal10[c(1:5,7,9,6,8,10)]

In [ ]:
load('HTAv2_figure5.RData')

#### Marker expression along pseudotime

In [ ]:
seurObj_CITE_4vs8$anno_CITE_4v5 <- seurObj_CITE_abconv$anno_CITE_4v5[match(seurObj_CITE_4vs8$sample_barcode, seurObj_CITE_abconv$sample_barcode)]

In [ ]:
options(repr.plot.width=9, repr.plot.height=7)

DimPlot(seurObj_CITE_4vs8, reduction = 'umap_wnn_4vs8', group.by = 'anno_CITE_4v5', shuffle=T, pt.size = 0.5, cols=pal12)

In [ ]:
max_CD4spt <- seurObj_CITE_4vs8$spt_spca_SP_CD4 %>% as.numeric %>% .[!is.na(.)] %>% max
max_CD8spt <- seurObj_CITE_4vs8$spt_spca_SP_CD8 %>% as.numeric %>% .[!is.na(.)] %>% max
max_CD4spt
max_CD8spt

In [ ]:
options(repr.plot.width=5, repr.plot.height=4.5)

pseudotimes_CD4 <- FeaturePlot(seurObj_CITE_4vs8, reduction = 'umap_wnn_4vs8', features = c('spt_spca_SP_CD4'),pt.size = 1, cols= rev(colorRampPalette(brewer.pal(11,'RdYlBu'))(max_CD4spt)), raster = T, raster.dpi = c(600,600))&theme_void()& labs(title = 'CD4 lineage pseudotime')

pseudotimes_CD4

In [ ]:
options(repr.plot.width=5, repr.plot.height=4.5)

pseudotimes_CD8 <- FeaturePlot(seurObj_CITE_4vs8, reduction = 'umap_wnn_4vs8', features = c('spt_spca_SP_CD8'),pt.size = 1, cols= rev(colorRampPalette(brewer.pal(11,'RdYlBu'))(max_CD8spt)), raster = T, raster.dpi = c(600,600))&theme_void()& labs(title = 'CD8 lineage pseudotime')

pseudotimes_CD8

In [ ]:
ADT2.df <- seurObj_CITE_4vs8_clean@assays$ADTdsb@data %>% t %>% data.frame
ADT2.df$spt_spca_SP_CD4 <- seurObj_CITE_4vs8_clean$spt_spca_SP_CD4
ADT2.df$spt_spca_SP_CD8 <- seurObj_CITE_4vs8_clean$spt_spca_SP_CD8
ADT2.df$anno_CITE_4v4 <- seurObj_CITE_4vs8_clean$anno_CITE_4v4
ADT2.df$anno_CITE_4v5 <- seurObj_CITE_4vs8_clean$anno_CITE_4v5
ADT2.df$dpt_rna_DP <- seurObj_CITE_4vs8_clean$dpt_rna_DP
ADT2.df$dpt_rank_rna_DP <- seurObj_CITE_4vs8_clean$dpt_rank_rna_DP
ADT2.df$pred_pseudotime_nhood_vdj <- seurObj_CITE_4vs8_clean$pred_pseudotime_nhood_vdj
ADT2.df$spt_spca_SP_Treg <- seurObj_CITE_4vs8_clean$spt_spca_SP_Treg
ADT2.df$spt_spca_SP_CD8aa <- seurObj_CITE_4vs8_clean$spt_spca_SP_CD8aa

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)

ggplot(ADT2.df, aes(y= anno_CITE_4v5, x=spt_spca_SP_CD4, color=anno_CITE_4v5))+
geom_jitter(size=0.5, alpha=0.3)+
scale_color_manual(values = pal10[3:10])+
theme_bw()+
labs(title='', y='', x='CD4 lineage sling pseudotime')&NoLegend()

ggplot(ADT2.df, aes(y= anno_CITE_4v5, x=spt_spca_SP_CD8, color=anno_CITE_4v5))+
geom_jitter(size=0.5, alpha=0.3)+
theme_bw()+
scale_color_manual(values = pal10[3:10])+
labs(title='', y='', x='CD8 lineage sling pseudotime')&NoLegend()

In [ ]:
genes_plotting2 <- c("TOX2", "ITM2A","EVL",'IL7R', 'PDE3B', 'RGS1',"MT2A", 'CD4', 'ZBTB7B', 'CLDN1',  'STAT5A','CD40LG', "TC2N", 'ANXA1', 'GPR183', "CD8A", "CD8B", 'RUNX3','FTL', 'STAG3', 'AOAH', 'PLAC8', 'LITAF', 'KLRK1')

In [ ]:
options(repr.plot.width=7, repr.plot.height=8)
DefaultAssay(seurObj_CITE_DPtoSP) <- 'RNA'

dotplot_markers <- DotPlot(seurObj_CITE_DPtoSP, group.by = 'anno_CITE_4v5', features=genes_plotting2)+
theme(axis.text.x = element_text(angle=45, hjust=1))+
geom_point(aes(size=pct.exp), shape = 21, colour="black", stroke=0.5) +
scale_color_distiller(palette = 'Spectral', direction = -1)+
labs(title = 'Selected markers')+
coord_flip()

dotplot_markers

In [ ]:
RNA.df <- seurObj_CITE_4vs8_clean@assays$RNA@data[rownames(seurObj_CITE_4vs8_clean@assays$RNA@data) %in% c(genes_plotting, 'CXCR3', 'CXCR4','CCR4', 'CCR6', 'CCR7', 'CCR8', 'CCR9', 'CCR10', 'CXCR5', 'CX3CR1'),] %>% t %>% data.frame
RNA.df$spt_spca_SP_CD4 <- seurObj_CITE_4vs8_clean$spt_spca_SP_CD4
RNA.df$spt_spca_SP_CD8 <- seurObj_CITE_4vs8_clean$spt_spca_SP_CD8
RNA.df$anno_CITE_4v5 <- seurObj_CITE_4vs8_clean$anno_CITE_4v5

In [ ]:
colors_rna <- c("CD4" = "red", "CD8A" = "blue", "CD8B" = "green3")
colors_protein <- c("CD4" = "red", "CD8" = "blue")

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

lineage4_prot <- ggplot(ADT2.df)+
geom_smooth(aes(y=CD4, x=spt_spca_SP_CD4, color='CD4'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD8, x=spt_spca_SP_CD4, color='CD8'), se=TRUE, method = 'gam')+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_protein)+
coord_cartesian(ylim = c(0,15))+
labs(title='', y='smoothed mean', x='')

lineage4_RNA <- ggplot(RNA.df)+
geom_smooth(aes(y=CD4, x=spt_spca_SP_CD4, color='CD4'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD8A, x=spt_spca_SP_CD4, color='CD8A'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD8B, x=spt_spca_SP_CD4, color='CD8B'), se=TRUE, method = 'gam')+
scale_color_manual(values = colors_rna)+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
labs(title='RNA expression', y='smoothed mean', x='')

lineage4_cells <- ggplot(RNA.df, aes(y= anno_CITE_4v5, x=spt_spca_SP_CD4, color=anno_CITE_4v5))+
geom_jitter_rast(size=0.5, alpha=0.5)+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12), panel.grid = element_blank(),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+scale_color_manual(values = pal10[3:10])+
labs(title='Cell types', y='', x='CD4 lineage sling pseudotime')&NoLegend()

options(repr.plot.width=5, repr.plot.height=3)
lineage4_RNA
lineage4_prot
options(repr.plot.width=6, repr.plot.height=3)
lineage4_cells

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

lineage8_prot <- ggplot(ADT2.df)+
geom_smooth(aes(y=CD4, x=spt_spca_SP_CD8, color='CD4'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD8, x=spt_spca_SP_CD8, color='CD8'), se=TRUE, method = 'gam')+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_protein)+
coord_cartesian(ylim = c(0,15))+
labs(title='Surface expression', y='smoothed mean', x='')

lineage8_RNA <- ggplot(RNA.df)+
geom_smooth(aes(y=CD4, x=spt_spca_SP_CD8, color='CD4'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD8A, x=spt_spca_SP_CD8, color='CD8A'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD8B, x=spt_spca_SP_CD8, color='CD8B'), se=TRUE, method = 'gam')+
scale_color_manual(values = colors_rna)+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
labs(title='RNA expression', y='smoothed mean', x='')

lineage8_cells <- ggplot(RNA.df, aes(y= anno_CITE_4v5, x=spt_spca_SP_CD8, color=anno_CITE_4v5))+
geom_jitter_rast(size=0.5, alpha=0.5)+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12), panel.grid = element_blank(),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = pal10[3:10])+
labs(title='Cell types', y='', x='CD8 lineage sling pseudotime')&NoLegend()

options(repr.plot.width=5, repr.plot.height=3)
lineage8_RNA
lineage8_prot
options(repr.plot.width=6, repr.plot.height=3)
lineage8_cells

In [ ]:
colors_receptors_prot <- c('CD183' ='red',
                           'CD194'='black',
                           'CD197'='green3',
                           'CD199'='dodgerblue')

colors_receptors_rna <- c('CXCR3' ='red',
                           'CCR4'='black',
                           'CCR7'='green3',
                           'CCR9'='dodgerblue',
                           'CXCR4'='purple')

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

receptors_lineage4_prot <- ggplot(ADT2.df)+
geom_smooth(aes(y=CD183, x=spt_spca_SP_CD4, color='CD183'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD194, x=spt_spca_SP_CD4, color='CD194'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD197, x=spt_spca_SP_CD4, color='CD197'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD199, x=spt_spca_SP_CD4, color='CD199'), se=TRUE, method = 'gam')+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_receptors_prot)+
coord_cartesian(ylim = c(0,12))+
theme(legend.key = element_rect(fill = "white"))+
labs(title='Surface expression', y='smoothed mean', x='', color='Receptor')

receptors_lineage4_rna <- ggplot(RNA.df)+
geom_smooth(aes(y=CXCR3, x=spt_spca_SP_CD4, color='CXCR3'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CCR4, x=spt_spca_SP_CD4, color='CCR4'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CCR7, x=spt_spca_SP_CD4, color='CCR7'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CCR9, x=spt_spca_SP_CD4, color='CCR9'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CXCR4, x=spt_spca_SP_CD4, color='CXCR4'), se=TRUE, method = 'gam')+
theme_bw()+
coord_cartesian(ylim = c(0,2.5))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_receptors_rna)+
theme(legend.key = element_rect(fill = "white"))+
labs(title='RNA expression', y='smoothed mean', x='', color='Receptor')


options(repr.plot.width=5, repr.plot.height=3)
receptors_lineage4_rna
receptors_lineage4_prot

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

receptors_lineage8_prot <- ggplot(ADT2.df)+
geom_smooth(aes(y=CD183, x=spt_spca_SP_CD8, color='CD183'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD194, x=spt_spca_SP_CD8, color='CD194'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD197, x=spt_spca_SP_CD8, color='CD197'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD199, x=spt_spca_SP_CD8, color='CD199'), se=TRUE, method = 'gam')+
theme_bw()+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_receptors_prot)+
coord_cartesian(ylim = c(0,12))+
theme(legend.key = element_rect(fill = "white"))+
labs(title='Surface expression', y='smoothed mean', x='', color='Receptor')

receptors_lineage8_rna <- ggplot(RNA.df)+
geom_smooth(aes(y=CXCR3, x=spt_spca_SP_CD8, color='CXCR3'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CCR4, x=spt_spca_SP_CD8, color='CCR4'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CCR7, x=spt_spca_SP_CD8, color='CCR7'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CCR9, x=spt_spca_SP_CD8, color='CCR9'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CXCR4, x=spt_spca_SP_CD8, color='CXCR4'), se=TRUE, method = 'gam')+
theme_bw()+
coord_cartesian(ylim = c(0,2.5))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_receptors_rna)+
theme(legend.key = element_rect(fill = "white"))+
labs(title='RNA expression', y='smoothed mean', x='', color='Receptor')


options(repr.plot.width=5, repr.plot.height=3)
receptors_lineage8_rna
receptors_lineage8_prot

In [ ]:
colors_protein2 <- c('CD1a'='red',
                    'CD44'='blue',
                    'CD45RA'='green3',
                    'CD45RO'='orange',
                    'CD27'='black',
                    'CD69'='purple',
                    'TCRab'='blue')

In [ ]:
markers_lineage4 <- ggplot(ADT2.df)+
geom_smooth(aes(y=CD1a, x=spt_spca_SP_CD4, color='CD1a'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD44, x=spt_spca_SP_CD4, color='CD44'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD45RA, x=spt_spca_SP_CD4, color='CD45RA'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD45RO, x=spt_spca_SP_CD4, color='CD45RO'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD27, x=spt_spca_SP_CD4, color='CD27'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD69, x=spt_spca_SP_CD4, color='CD69'), se=TRUE, method = 'gam')+
theme_bw()+
coord_cartesian(ylim = c(0,40))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_protein2)+
theme(legend.key = element_rect(fill = "white"))+
labs(title='Surface expression on CD4 lineage', y='smoothed mean', x='', color='Receptor')

options(repr.plot.width=5, repr.plot.height=3)

markers_lineage4

In [ ]:
options(repr.plot.width=6, repr.plot.height=4)

markers_lineage8 <- ggplot(ADT2.df)+
geom_smooth(aes(y=CD1a, x=spt_spca_SP_CD8, color='CD1a'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD44, x=spt_spca_SP_CD8, color='CD44'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD45RA, x=spt_spca_SP_CD8, color='CD45RA'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD45RO, x=spt_spca_SP_CD8, color='CD45RO'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD27, x=spt_spca_SP_CD8, color='CD27'), se=TRUE, method = 'gam')+
geom_smooth(aes(y=CD69, x=spt_spca_SP_CD8, color='CD69'), se=TRUE, method = 'gam')+
theme_bw()+
coord_cartesian(ylim = c(0,40))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+
scale_color_manual(values = colors_protein2)+
theme(legend.key = element_rect(fill = "white"))+
labs(title='Surface expression on CD8 lineage', y='smoothed mean', x='', color='Receptor')

options(repr.plot.width=5, repr.plot.height=3)

markers_lineage8

#### Visium-derived CMA

In [ ]:
meta_cma <- read.csv('../data/cite_paed_hyper_mapping_v4.csv')

In [ ]:
#need to adjust barcodes to match CITEseq data
meta_cma$sample_barcode <- gsub("-[0-9]+", "", meta_cma$sample_barcode)

In [ ]:
table(seurObj_CITE_DPtoSP$sample_barcode %in% meta_cma$sample_barcode)

In [ ]:
meta_cma_subs <- subset(meta_cma, sample_barcode %in% seurObj_CITE_DPtoSP$sample_barcode)

In [ ]:
seurObj_CITE_DPtoSP_subs <- subset(seurObj_CITE_DPtoSP, sample_barcode %in% meta_cma_subs$sample_barcode)

In [ ]:
matching <- match(seurObj_CITE_DPtoSP_subs$sample_barcode, meta_cma_subs$sample_barcode)

In [ ]:
seurObj_CITE_DPtoSP_subs$mean_hyper_cma_exp <- as.numeric(meta_cma_subs$mean_hyper_cma_exp[matching])

In [ ]:
options(repr.plot.width=5, repr.plot.height=3.5)

ggplot(seurObj_CITE_DPtoSP_subs@meta.data, aes(x=anno_CITE_4v5, y=mean_hyper_cma_exp, fill=anno_CITE_4v5))+
geom_boxplot(outlier.shape = NA)+
geom_jitter_rast(alpha=0.2, size=0.1, width=0.1, color='grey30')+
theme_bw()+
scale_y_reverse()+
theme(axis.text.x = element_text(angle=45, hjust=1), axis.text.y = element_text(size = 12), panel.grid = element_blank(),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+scale_fill_manual(values = pal10v2)+
NoLegend()+
labs(x='')

In [ ]:
matching2 <- match(seurObj_CITE_4vs8_clean$sample_barcode, meta_cma$sample_barcode)

In [ ]:
seurObj_CITE_4vs8_clean$mean_hyper_cma_exp <- as.numeric(meta_cma$mean_hyper_cma_exp[matching2])

In [ ]:
genes_plotting <- c("TOP2A", 'MKI67', "RAG1", "RAG2", "DNTT", 'LIG4', 'RORC', 'BCL2L1', 'NR4A1','SATB1',"TOX2", "ITM2A","EVL",'IL7R', 'STAT1', 'PDE3B', 'RGS1',"MT2A", "IFI44L", 'CD4', 'ZBTB7B','CLDN1', 'HPGD',  'STAT5A','CD40LG', "ICOS", "TC2N", 'ANXA1', 'GIMAP5', 'GPR183', "CD8A", "CD8B", 'RUNX3','FTL', 'STAG3', 'AOAH', 'CLIC3', 'PLAC8', 'LITAF', 'KLRK1')

In [ ]:
# add mapping info to data frame
RNA.df$mean_hyper_cma_exp <- seurObj_CITE_4vs8_clean$mean_hyper_cma_exp

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

cma_lineage4 <- ggplot(RNA.df, aes(y=mean_hyper_cma_exp, x=spt_spca_SP_CD4))+
geom_jitter_rast(aes(color=anno_CITE_4v5), size=0.1, alpha=0.1)+
geom_smooth(se=T, method = 'gam', color='black', size=0.5)+
theme_bw()+
geom_hline(yintercept = 0, linetype=2, color='grey50')+
scale_y_reverse()+
coord_cartesian(ylim = c(0.65,-0.55))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
labs(title='Visium-predicted CMA for CD4 lineage', y='mean cluster exp(CMA)', x='CD4 lineage pseudotime')+
scale_color_manual(values = pal10[3:10])+
scale_x_continuous(breaks=seq(0,10.5,1.5))&NoLegend()

cma_lineage4

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

cma_lineage8 <- ggplot(RNA.df, aes(y=mean_hyper_cma_exp, x=spt_spca_SP_CD8))+
geom_jitter_rast(aes(color=anno_CITE_4v5), size=0.1, alpha=0.1)+
geom_smooth(se=T, method = 'gam', color='black',  size=0.5)+
geom_hline(yintercept = 0, linetype=2, color='grey50')+
theme_bw()+
scale_y_reverse()+
coord_cartesian(ylim = c(0.65,-0.55))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
labs(title='Visium-predicted CMA for CD8 lineage', y='mean cluster exp(CMA)', x='CD8 lineage pseudotime')+
scale_color_manual(values = pal10[3:10])+
scale_x_continuous(breaks=seq(0,12.5,2.5))&NoLegend()

cma_lineage8

For better visualisation of distribution we can calculate and plot medians + quantiles per annotation.

In [ ]:
DPtoSP_cells2 <-c('DP_pos_sel','DP_4hi8lo', 'SP_CD4_immature', 'SP_CD4_semimature','SP_CD4_mature','SP_CD8_immature','SP_CD8_semimature', 'SP_CD8_mature')

In [ ]:
meta_visium_summary <- RNA.df %>%
    subset(anno_CITE_4v5 %in% DPtoSP_cells2) %>% 
    group_by(anno_CITE_4v5) %>% summarise(
    visium_spt_CD4_median = median(spt_spca_SP_CD4, na.rm = T),                  
    visium_spt_CD4_q05 = quantile(spt_spca_SP_CD4, probs = 0.05, na.rm = T),
    visium_spt_CD4_q95 = quantile(spt_spca_SP_CD4, probs = 0.95, na.rm = T),
    visium_spt_CD8_median = median(spt_spca_SP_CD8, na.rm = T),                  
    visium_spt_CD8_q05 = quantile(spt_spca_SP_CD8, probs = 0.05, na.rm = T),
    visium_spt_CD8_q95 = quantile(spt_spca_SP_CD8, probs = 0.95, na.rm = T))

#need to calculate CMA means for lineages separately because for the hilo cells not all will be included in the pseudotime of both lineages

meta_visium_summary <- RNA.df %>%
    subset(anno_CITE_4v5 %in% DPtoSP_cells2 & !is.na(spt_spca_SP_CD8)) %>% 
    group_by(anno_CITE_4v5) %>% summarise(
    visium_cma_median_CD8 = median(mean_hyper_cma_exp, na.rm = T),                  
    visium_cma_q05_CD8 = quantile(mean_hyper_cma_exp, probs = 0.05, na.rm = T),
    visium_cma_q95_CD8 = quantile(mean_hyper_cma_exp, probs = 0.95, na.rm = T)) %>%
    left_join(meta_visium_summary, .)

meta_visium_summary <- RNA.df %>%
    subset(anno_CITE_4v5 %in% DPtoSP_cells2 & !is.na(spt_spca_SP_CD4)) %>% 
    group_by(anno_CITE_4v5) %>% summarise(
    visium_cma_median_CD4 = median(mean_hyper_cma_exp, na.rm = T),                  
    visium_cma_q05_CD4 = quantile(mean_hyper_cma_exp, probs = 0.05, na.rm = T),
    visium_cma_q95_CD4 = quantile(mean_hyper_cma_exp, probs = 0.95, na.rm = T)) %>%
    left_join(meta_visium_summary, .)

meta_visium_summary$anno_CITE_4v5 <- factor(meta_visium_summary$anno_CITE_4v5, levels=DPtoSP_cells2)
meta_visium_summary

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

cma_lineage4_v2 <- ggplot()+
geom_jitter_rast(data=RNA.df, aes(y=mean_hyper_cma_exp, x=spt_spca_SP_CD4,color=anno_CITE_4v5), size=0.1, alpha=0.4)+
geom_errorbarh(data = meta_visium_summary,aes(xmin = visium_spt_CD4_q05,xmax = visium_spt_CD4_q95,y = visium_cma_median_CD4), height=0.03, size=0.2, color='black') + 
geom_errorbar(data = meta_visium_summary,aes(ymin = visium_cma_q05_CD4,ymax = visium_cma_q95_CD4,x = visium_spt_CD4_median), width=0.2, size=0.2, color='black')+
geom_point(data = meta_visium_summary,aes(x=visium_spt_CD4_median, y=visium_cma_median_CD4, fill = anno_CITE_4v5), size=3, shape=21, color='black') +
theme_bw()+
scale_y_reverse()+
geom_hline(yintercept = 0, linetype=2, color='grey50')+
coord_cartesian(ylim = c(0.65,-0.55))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
labs(title='Visium-predicted CMA for CD4 lineage', y='mean cluster exp(CMA)', x='CD4 lineage pseudotime')+
scale_color_manual(values = pal10[3:10])+
scale_fill_manual(values = pal10v2[3:10])+
scale_x_continuous(breaks=seq(0,10.5,1.5))&NoLegend()

cma_lineage4_v2

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

cma_lineage8_v2 <- ggplot()+
geom_jitter_rast(data=RNA.df, aes(y=mean_hyper_cma_exp, x=spt_spca_SP_CD8,color=anno_CITE_4v5), size=0.1, alpha=0.4)+
geom_errorbarh(data = meta_visium_summary,aes(xmin = visium_spt_CD8_q05,xmax = visium_spt_CD8_q95,y = visium_cma_median_CD8), height=0.03, size=0.2, color='black') + 
geom_errorbar(data = meta_visium_summary,aes(ymin = visium_cma_q05_CD8,ymax = visium_cma_q95_CD8,x = visium_spt_CD8_median), width=0.2, size=0.2, color='black')+
geom_point(data = meta_visium_summary,aes(x=visium_spt_CD8_median, y=visium_cma_median_CD8, fill = anno_CITE_4v5), size=3, shape=21, color='black') +
theme_bw()+
scale_y_reverse()+
geom_hline(yintercept = 0, linetype=2, color='grey50')+
coord_cartesian(ylim = c(0.65,-0.55))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
labs(title='Visium-predicted CMA for CD8 lineage', y='mean cluster exp(CMA)', x='CD8 lineage pseudotime')+
scale_color_manual(values = pal10[3:10])+
scale_fill_manual(values = pal10v2[3:10])+
scale_x_continuous(breaks=seq(0,12.5,2.5))&NoLegend()

cma_lineage8_v2

#### IBEX mapping

CITE-seq annotations and pseudotime have been transferred to segmented cells from IBEX using KNN matching. We can plot these cells in the same way as the CITE-seq cells that have been Visium-mapped.

In [ ]:
meta_IBEX_hybrid <- read.csv('../data/out_cite_ibex_cite_T_only_th_5_KNN5_peudo.csv')

In [ ]:
meta_IBEX_comb <- meta_IBEX_hybrid[,c(1,20,21,38,25:28,33:37)]
colnames(meta_IBEX_comb)[5:13] <- paste0('hybrid_',colnames(meta_IBEX_comb)[5:13])

In [ ]:
DPtoSP_cells <- names(table(meta_IBEX_comb$hybrid_anno_CITE_4v6))[c(12,14:16, 20, 22, 32:36,38)]

In [ ]:
meta_IBEX_comb_T <- subset(meta_IBEX_comb, hybrid_anno_CITE_4v6 %in% DPtoSP_cells | rna_anno_CITE_4v6 %in% DPtoSP_cells)
meta_IBEX_comb_T$hybrid_anno_CITE_4v6 <- factor(meta_IBEX_comb_T$hybrid_anno_CITE_4v6, levels=DPtoSP_cells)

In [ ]:
meta_IBEX_comb_T2_hybrid <- subset(meta_IBEX_comb_T, hybrid_anno_CITE_4v6 %in% DPtoSP_cells2)
meta_IBEX_comb_T2_hybrid$hybrid_anno_CITE_4v6 <- factor(meta_IBEX_comb_T2_hybrid$hybrid_anno_CITE_4v6, levels=DPtoSP_cells2)

In [ ]:
options(repr.plot.width=5.8, repr.plot.height=4)

scatter_pseudtimesCD4_hybrid <- ggplot(meta_IBEX_comb_T2_hybrid, aes(x=hybrid_spt_spca_SP_CD4, y=rna_spt_spca_SP_CD4, color=hybrid_anno_CITE_4v6))+
geom_point_rast(size=0.2, alpha=0.7)+
geom_abline(linetype=2, color='grey50')+
scale_color_manual(values = pal10v2[3:10])+
theme_bw()+
scale_x_continuous(breaks=seq(0,10.5,1.5))+
scale_y_continuous(breaks=seq(0,10.5,1.5))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
            axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
guides(colour = guide_legend(override.aes = list(size=3)))+
labs(title='CD4 lineage pseudotime predicted from CITE-seq data', y= 'Pseudotime predicted based on RNA', x='Pseudotime predicted based on RNA/protein', color='Cell annotation\npredicted from CITE-seq\nRNA/protein data')

scatter_pseudtimesCD4_hybrid

In [ ]:
options(repr.plot.width=5.8, repr.plot.height=4)

scatter_pseudtimesCD8_hybrid <- ggplot(meta_IBEX_comb_T2_hybrid, aes(x=hybrid_spt_spca_SP_CD8, y=rna_spt_spca_SP_CD8, color=hybrid_anno_CITE_4v6))+
geom_point_rast(size=0.2, alpha=0.9)+
geom_abline(linetype=2, color='grey50')+
scale_color_manual(values = pal10v2[3:10])+
theme_bw()+
scale_x_continuous(breaks=seq(0,12.5,2.5))+
scale_y_continuous(breaks=seq(0,12.5,2.5))+
theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 12),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
            axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
guides(colour = guide_legend(override.aes = list(size=3)))+
labs(title='CD8 lineage pseudotime predicted from CITE-seq data', y= 'Pseudotime predicted based on RNA', x='Pseudotime predicted based on RNA/protein', color='Cell annotation\npredicted from CITE-seq\nRNA/protein data')

scatter_pseudtimesCD8_hybrid

In [ ]:
fraction_cuttoff <- 0.5 #use this to exclude cells with low annotation certainty

meta_IBEX_comb_T_summary <- meta_IBEX_comb_T %>%
    subset(hybrid_anno_CITE_4v6 %in% DPtoSP_cells2 & hybrid_anno_CITE_4v6_fraction >= fraction_cuttoff) %>% 
    group_by(hybrid_anno_CITE_4v6) %>% summarise(
    hybrid_spt_CD4_median = median(hybrid_spt_spca_SP_CD4, na.rm = T),                  
    hybrid_spt_CD4_q05 = quantile(hybrid_spt_spca_SP_CD4, probs = 0.05, na.rm = T),
    hybrid_spt_CD4_q95 = quantile(hybrid_spt_spca_SP_CD4, probs = 0.95, na.rm = T),
    hybrid_spt_CD8_median = median(hybrid_spt_spca_SP_CD8, na.rm = T),                  
    hybrid_spt_CD8_q05 = quantile(hybrid_spt_spca_SP_CD8, probs = 0.05, na.rm = T),
    hybrid_spt_CD8_q95 = quantile(hybrid_spt_spca_SP_CD8, probs = 0.95, na.rm = T))

#need to calculate CMA means for lineages separately because for the hilo cells not all will be included in the pseudotime of both lineages

meta_IBEX_comb_T_summary <- meta_IBEX_comb_T %>%
    subset(hybrid_anno_CITE_4v6 %in% DPtoSP_cells2 & hybrid_anno_CITE_4v6_fraction >= fraction_cuttoff & !is.na(hybrid_spt_spca_SP_CD4)) %>% 
    group_by(hybrid_anno_CITE_4v6) %>% summarise(
    hybrid_cma_median_CD4 = median(cma_v2_exp2, na.rm = T),                  
    hybrid_cma_q05_CD4 = quantile(cma_v2_exp2, probs = 0.05, na.rm = T),
    hybrid_cma_q95_CD4 = quantile(cma_v2_exp2, probs = 0.95, na.rm = T)) %>%
    left_join(meta_IBEX_comb_T_summary_hybrid, .)

meta_IBEX_comb_T_summary <- meta_IBEX_comb_T %>%
    subset(hybrid_anno_CITE_4v6 %in% DPtoSP_cells2 & hybrid_anno_CITE_4v6_fraction >= fraction_cuttoff & !is.na(hybrid_spt_spca_SP_CD8)) %>% 
    group_by(hybrid_anno_CITE_4v6) %>% summarise(
    hybrid_cma_median_CD8 = median(cma_v2_exp2, na.rm = T),                  
    hybrid_cma_q05_CD8 = quantile(cma_v2_exp2, probs = 0.05, na.rm = T),
    hybrid_cma_q95_CD8 = quantile(cma_v2_exp2, probs = 0.95, na.rm = T)) %>%
    left_join(meta_IBEX_comb_T_summary_hybrid, .)

        
meta_IBEX_comb_T_summary$hybrid_anno_CITE_4v6 <- factor(meta_IBEX_comb_T_summary$hybrid_anno_CITE_4v6, levels=DPtoSP_cells2)


In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

IBEX_CD4_pseu2 <- ggplot() + 
    geom_point(data=meta_IBEX_comb_T2_hybrid, aes(x=hybrid_spt_spca_SP_CD4, y=cma_v2_exp2, color=hybrid_anno_CITE_4v6), size=0.1, alpha=0.2) + 
    geom_errorbarh(data = meta_IBEX_comb_T_summary,aes(xmin = hybrid_spt_CD4_q05,xmax = hybrid_spt_CD4_q95,y = hybrid_cma_median_CD4), color='black', height=0.03, size=0.2) + 
    geom_errorbar(data = meta_IBEX_comb_T_summary,aes(ymin = hybrid_cma_q05_CD4,ymax = hybrid_cma_q95_CD4,x = hybrid_spt_CD4_median), color='black', width=0.2, size=0.2)+
    geom_point(data = meta_IBEX_comb_T_summary,aes(x=hybrid_spt_CD4_median, y=hybrid_cma_median_CD4, fill = hybrid_anno_CITE_4v6), size=3, shape=23, color='black') +
    scale_fill_manual(values = pal10v2[3:10])+
    scale_y_reverse()+
    coord_cartesian(ylim = c(0.65,-0.55), xlim = c(0,11.2))+
    scale_colour_manual(values = pal10v2[3:10])+
    theme_bw()+
    theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
            axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
    theme(legend.key = element_rect(fill = "white"))+
    labs(title='IBEX (RNA/protein) CMA for CD4 lineage', y='mean exp(CMA)', x='CD4 lineage pseudotime')+
    geom_hline(yintercept = 0, linetype=2, color='grey50')+
    scale_x_continuous(breaks=seq(0,10.5,1.5))&NoLegend()


IBEX_CD4_pseu2

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

IBEX_CD8_pseu2 <- ggplot() + 
    geom_point(data=meta_IBEX_comb_T2_hybrid, aes(x=hybrid_spt_spca_SP_CD8, y=cma_v2_exp2, color=hybrid_anno_CITE_4v6), size=0.1, alpha=0.2) + 
    geom_errorbarh(data = meta_IBEX_comb_T_summary,aes(xmin = hybrid_spt_CD8_q05,xmax = hybrid_spt_CD8_q95,y = hybrid_cma_median_CD8), colour = "black", height=0.03, size=0.2) + 
    geom_errorbar(data = meta_IBEX_comb_T_summary,aes(ymin = hybrid_cma_q05_CD8,ymax = hybrid_cma_q95_CD8,x = hybrid_spt_CD8_median), colour = "black", width=0.2, size=0.2)+
    geom_point(data = meta_IBEX_comb_T_summary,aes(x=hybrid_spt_CD8_median, y=hybrid_cma_median_CD8, fill = hybrid_anno_CITE_4v6), size=3, shape=23, color='black') +
    scale_fill_manual(values = pal10v2[3:10])+
    scale_y_reverse()+
    coord_cartesian(ylim = c(0.65,-0.55))+
    scale_colour_manual(values = pal10v2[3:10])+
    theme_bw()+
    theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
            axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
    theme(legend.key = element_rect(fill = "white"))+
    labs(title='IBEX (RNA/protein) CMA for CD8 lineage', y='mean exp(CMA)', x='CD8 lineage pseudotime')+
    geom_hline(yintercept = 0, linetype=2, color='grey50')+
    scale_x_continuous(breaks=seq(0,12.5,2.5))&NoLegend()

IBEX_CD8_pseu2

#### Comparison of Visium and IBEX

In [ ]:
meta_spatial_summary <- left_join(meta_IBEX_comb_T_summary, meta_visium_summary, join_by(hybrid_anno_CITE_4v6==anno_CITE_4v5))
meta_spatial_summary

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

comp_pseu_CD4 <- ggplot() + 
   geom_errorbarh(data = meta_spatial_summary,aes(xmin = visium_spt_CD4_q05,xmax = visium_spt_CD4_q95,y = visium_cma_median_CD4,colour = hybrid_anno_CITE_4v6), height=0.03) + 
   geom_errorbar(data = meta_spatial_summary,aes(ymin = visium_cma_q05_CD4,ymax = visium_cma_q95_CD4,x = visium_spt_CD4_median,colour = hybrid_anno_CITE_4v6), width=0.2)+
   geom_errorbarh(data = meta_spatial_summary,aes(xmin = hybrid_spt_CD4_q05,xmax = rna_spt_CD4_q95,y = hybrid_cma_median_CD4,colour = hybrid_anno_CITE_4v6), height=0.03, linetype=2) + 
    geom_errorbar(data = meta_spatial_summary,aes(ymin = hybrid_cma_q05_CD4,ymax = hybrid_cma_q95_CD4,x = hybrid_spt_CD4_median,colour = hybrid_anno_CITE_4v6), width=0.2, linetype=2)+
    geom_point(data = meta_spatial_summary,aes(x=visium_spt_CD4_median, y=visium_cma_median_CD4, fill = hybrid_anno_CITE_4v6), size=5, shape=21, color='black') +
   geom_point(data = meta_spatial_summary,aes(x=hybrid_spt_CD4_median, y=hybrid_cma_median_CD4, fill = hybrid_anno_CITE_4v6), size=5, shape=23, color='black') +
    scale_fill_manual(values = pal10v2[3:10])+
    scale_colour_manual(values = pal10v2[3:10])+
    scale_y_reverse()+
    coord_cartesian(ylim = c(0.65,-0.55))+
    theme_bw()+
    theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
            axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
    theme(legend.key = element_rect(fill = "white"))+
    labs(title='CMA for CD4 lineage', y='mean exp(CMA)', x='CD4 lineage pseudotime')+
    geom_hline(yintercept = 0, linetype=2, color='grey50')+
    scale_x_continuous(breaks=seq(0,10.5,1.5))&NoLegend()

comp_pseu_CD4

In [ ]:
options(repr.plot.width=4.5, repr.plot.height=3)

comp_pseu_CD8 <- ggplot() + 
   geom_errorbarh(data = meta_spatial_summary,aes(xmin = visium_spt_CD8_q05,xmax = visium_spt_CD8_q95,y = visium_cma_median_CD8,colour = hybrid_anno_CITE_4v6), height=0.03) + 
   geom_errorbar(data = meta_spatial_summary,aes(ymin = visium_cma_q05_CD8,ymax = visium_cma_q95_CD8,x = visium_spt_CD8_median,colour = hybrid_anno_CITE_4v6), width=0.2)+
   geom_errorbarh(data = meta_spatial_summary,aes(xmin = hybrid_spt_CD8_q05,xmax = rna_spt_CD8_q95,y = hybrid_cma_median_CD8,colour = hybrid_anno_CITE_4v6), height=0.03, linetype=2) + 
    geom_errorbar(data = meta_spatial_summary,aes(ymin = hybrid_cma_q05_CD8,ymax = hybrid_cma_q95_CD8,x = hybrid_spt_CD8_median,colour = hybrid_anno_CITE_4v6), width=0.2, linetype=2)+
    geom_point(data = meta_spatial_summary,aes(x=visium_spt_CD8_median, y=visium_cma_median_CD8, fill = hybrid_anno_CITE_4v6), size=5, shape=21, color='black') +
   geom_point(data = meta_spatial_summary,aes(x=hybrid_spt_CD8_median, y=hybrid_cma_median_CD8, fill = hybrid_anno_CITE_4v6), size=5, shape=23, color='black') +
     scale_fill_manual(values = pal10v2[3:10])+
    scale_colour_manual(values = pal10v2[3:10])+
    scale_y_reverse()+
    coord_cartesian(ylim = c(0.65,-0.55))+
    theme_bw()+
    theme(axis.text.x = element_text(size=12), axis.text.y = element_text(size = 14),
           panel.border = element_blank(), axis.line.x = element_line(size = 0.5, linetype = "solid", colour = "black"),
            axis.line.y = element_line(size = 0.5, linetype = "solid", colour = "black"))+theme(legend.key = element_rect(fill = "white"))+
    theme(legend.key = element_rect(fill = "white"))+
    labs(title='CMA for CD8 lineage', y='mean exp(CMA)', x='CD8 lineage pseudotime')+
    geom_hline(yintercept = 0, linetype=2, color='grey50')+
    scale_x_continuous(breaks=seq(0,12.5,2.5))&NoLegend()

comp_pseu_CD8